# Pair Trading BCHUSDT-BTCUSDT

This notebook uses a cryptocurrency trading strategy for BCHUSDT-BTCUSDT and search for the perido window with the heigth return during the timeframe.

Data comes from Biancne and is hourly data between 2019-11-28 to 2020-10-19.

References:

https://www.backtrader.com/docu/pandas-datafeed/pandas-datafeed/

https://github.com/mementum/backtrader/blob/master/contrib/samples/pair-trading/pair-trading.py

https://www.backtrader.com/blog/posts/2015-08-12-observers-and-statistics/observers-and-statistics/



In [8]:
import pandas as pd

In [9]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

In [4]:
%matplotlib inline
# import packages
from binance.client import Client
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sklearn as skl
import pytz
import statsmodels.api as sm
#import zipline
import pathlib

# get api keys for binance 
import os
API_PUBLIC = '<>'
API_SECRET = '<>'
CSV_PATH_PREFIX = os.environ.get("PATH_TO_PROJECT")

crypto_pairs = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BCHUSDT', 'BNBUSDT']

# iteratively loop through cryptocurrencies in crypto_pairs and download them into dataframes
# set granularity of time series
timeperiod = '1h' # hourly
client = Client(API_PUBLIC, API_SECRET)
timestamp = []
for pair in crypto_pairs:
    # determine first available price point
    timestamp.append(client._get_earliest_valid_timestamp(pair, timeperiod)) 
# assign first available price point to be be the latest in time
timestamp = max(timestamp)

startdate = timestamp #"1 Jan, 2019"
df = {} # create a dictionary that will eventually contain currency dataframes
for pair in crypto_pairs:
    klines = client.get_historical_klines(pair, timeperiod, startdate)
    
    # delete unwanted data - just keep date, open, high, low, close
    for line in klines:
        del line[6:]
        
    # cast into dataframe and index with date
    df[pair] = pd.DataFrame(klines, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df[pair]['date'] = pd.to_datetime(df[pair]['date'],unit='ms')
    df[pair].set_index('date', inplace=True)
    
    # previous testing showed that the columns of df are objects. convert to float
    df[pair] = df[pair].astype(float)
    
for key in df:
    df[key].to_csv('hourly/'+key + '.csv')
    print(key)

# import data
df_hourly = {}
for key in os.listdir('hourly'):
    print(key[:-4]) # the weird slicing is to drop the .csv extension
    df_hourly[key[:-4]] = pd.read_csv('hourly/' + key)
    df_hourly[key[:-4]]['date'] = pd.to_datetime(df_hourly[key[:-4]]['date'])# ,unit='ms')
    df_hourly[key[:-4]].set_index('date', inplace=True)

BCHUSDT
BNBUSDT
ETHUSDT
BTCUSDT
XRPUSDT


In [5]:
df = df_hourly

In [46]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import os

import argparse
import datetime

# The above could be sent to an independent module
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

#create datafeed class for pandas dataframes
class PandasData(btfeeds.feed.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
    )


class PairTradingStrategy(bt.Strategy):
    params = dict(
        period=10,
        stake=10,
        qty1=0,
        qty2=0,
        printout=True,
        upper=2.1,
        lower=-2.1,
        up_medium=0.5,
        low_medium=-0.5,
        status=0,
        portfolio_value=10000,
        name1='none',
        name2='none'
    )

    def start(self):
        #record results
        if not os.path.exists('mystats_hour.csv'):
            self.mystats = open('mystats_hour.csv', 'w')
            self.mystats.write('date,pair1,pair2,portfolio,period,status,zscore,pair1_pos,pair2_pos,order_flag\n')
        else:
            self.mystats = open('mystats_hour.csv', 'a')
            
        if not os.path.exists('endstats_hour.csv'):
            self.mystats2 = open('endstats_hour.csv', 'w')
            self.mystats2.write('pair1,pair2,portfolio,period\n')
        else:
            self.mystats2 = open('endstats_hour.csv', 'a')

        
    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.qty1 = self.p.qty1
        self.qty2 = self.p.qty2
        self.upper_limit = self.p.upper
        self.lower_limit = self.p.lower
        self.up_medium = self.p.up_medium
        self.low_medium = self.p.low_medium
        self.status = self.p.status
        self.portfolio_value = self.p.portfolio_value
        self.pair1 = self.p.name1
        self.pair2 = self.p.name2

        # Signals performed with PD.OLS :
        self.transform = btind.OLS_TransformationN(self.data0, self.data1,
                                                   period=self.p.period)
        self.zscore = self.transform.zscore

    def next(self):
        
        

        if self.orderid:
            self.mystats.write(self.data.datetime.datetime(0).strftime('%Y-%m-%dT%H:%M:%S'))
            self.mystats.write(',{}'.format(self.pair1))

            self.mystats.write(',{}'.format(self.pair2))
            self.mystats.write(',{}'.format(self.broker.getvalue()))
            self.mystats.write(',{}'.format(self.p.period))
            self.mystats.write(',{}'.format(self.status))
            self.mystats.write(',{}'.format(self.zscore[0]))
            self.mystats.write(',{}'.format(self.getposition(self.data0).size))
            self.mystats.write(',{}'.format(self.getposition(self.data1).size))
            self.mystats.write(',{}'.format(True))
            self.mystats.write('\n')
            return  # if an order is active, no new orders are allowed

        # Step 2: Check conditions for SHORT & place the order
        # Checking the condition for SHORT
        if (self.zscore[0] > self.upper_limit) and (self.status != 1):

            # Calculating the number of shares for each stock
            value = 0.5 * self.portfolio_value  # Divide the cash equally
            x = int(value / (self.data0.close))  # Find the number of shares for Stock1
            y = int(value / (self.data1.close))  # Find the number of shares for Stock2
            print('x + self.qty1 is', x + self.qty1)
            print('y + self.qty2 is', y + self.qty2)

            # Placing the order
            self.log('SELL CREATE %s, price = %.2f, qty = %d' % (name1, self.data0.close[0], x + self.qty1))
            self.sell(data=self.data0, size=(x + self.qty1))  # Place an order for buying y + qty2 shares
            self.log('BUY CREATE %s, price = %.2f, qty = %d' % (name2, self.data1.close[0], y + self.qty2))
            self.buy(data=self.data1, size=(y + self.qty2))  # Place an order for selling x + qty1 shares

            # Updating the counters with new value
            self.qty1 = x  # The new open position quantity for Stock1 is x shares
            self.qty2 = y  # The new open position quantity for Stock2 is y shares

            self.status = 1  # The current status is "short the spread"

            # Step 3: Check conditions for LONG & place the order
            # Checking the condition for LONG
        elif (self.zscore[0] < self.lower_limit) and (self.status != 2):
            # Calculating the number of shares for each stock
            value = 0.5 * self.portfolio_value  # Divide the cash equally
            x = int(value / (self.data0.close))  # Find the number of shares for Stock1
            y = int(value / (self.data1.close))  # Find the number of shares for Stock2
            print('x + self.qty1 is', x + self.qty1)
            print('y + self.qty2 is', y + self.qty2)

            # Place the order
            self.log('BUY CREATE %s, price = %.2f, qty = %d' % (name1, self.data0.close[0], x + self.qty1))
            self.buy(data=self.data0, size=(x + self.qty1))  # Place an order for buying x + qty1 shares
            self.log('SELL CREATE %s, price = %.2f, qty = %d' % (name2, self.data1.close[0], y + self.qty2))
            self.sell(data=self.data1, size=(y + self.qty2))  # Place an order for selling y + qty2 shares

            # Updating the counters with new value
            self.qty1 = x  # The new open position quantity for Stock1 is x shares
            self.qty2 = y  # The new open position quantity for Stock2 is y shares
            self.status = 2  # The current status is "long the spread"


        # Step 4: Check conditions for No Trade
        # If the z-score is within the two bounds, close all
        
        elif (self.zscore[0] < self.up_medium and self.zscore[0] > self.low_medium and self.status != 0):
            self.log('CLOSE Position %s, price = %.2f, qty = %d' % (name1, self.data0.close[0],self.getposition(self.data0).size))
            self.close(self.data0)
            self.log('CLOSE Position %s, price = %.2f, qty = %d' % (name2, self.data1.close[0],self.getposition(self.data1).size))
            self.close(self.data1)
            self.status = 0
            
        self.mystats.write(self.data.datetime.datetime(0).strftime('%Y-%m-%dT%H:%M:%S'))
        self.mystats.write(',{}'.format(self.pair1))
        
        self.mystats.write(',{}'.format(self.pair2))
        self.mystats.write(',{}'.format(self.broker.getvalue()))
        self.mystats.write(',{}'.format(self.p.period))
        self.mystats.write(',{}'.format(self.status))
        self.mystats.write(',{}'.format(self.zscore[0]))
        self.mystats.write(',{}'.format(self.getposition(self.data0).size))
        self.mystats.write(',{}'.format(self.getposition(self.data1).size))
        self.mystats.write(',{}'.format(False))
        self.mystats.write('\n')
        

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')
        self.mystats2.write('{}'.format(self.pair1))
        
        self.mystats2.write(',{}'.format(self.pair2))
        self.mystats2.write(',{}'.format(self.broker.getvalue()))
        self.mystats2.write(',{}'.format(self.p.period))
        self.mystats2.write('\n')
        self.mystats.close()
        self.mystats2.close()


def runstrategy(name1,name2,period,cash):


    # Create a cerebro
    cerebro = bt.Cerebro()

    # Create the 1st data
    data0 = bt.feeds.PandasData(dataname=df[name1])

    # Add the 1st data to cerebro
    cerebro.adddata(data0)

    # Create the 2nd data
    data1 = bt.feeds.PandasData(dataname=df[name2])

    # Add the 2nd data to cerebro
    cerebro.adddata(data1)

    # Add the strategy
    cerebro.addstrategy(PairTradingStrategy,
                        period=period,
                        stake=10,name1=name1,name2=name2,status=0,portfolio_value=cash)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcash(cash)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcommission(commission=0.0)

    # And run it
    cerebro.run()

In [48]:
# search for results for different periods and pairs
# note order of the pair matters
for period in range(264,288,24):
    for name1 in ['BCHUSDT']:
        for name2 in ['BTCUSDT']:
            if name1==name2:
                None
            else:
                print(period,name1,name2)
                runstrategy(name1,name2,period,100000)

264 BCHUSDT BTCUSDT
x + self.qty1 is 239
y + self.qty2 is 6
2019-12-09T18:00:00, BUY CREATE BCHUSDT, price = 208.68, qty = 239
2019-12-09T18:00:00, SELL CREATE BTCUSDT, price = 7444.57, qty = 6
2019-12-09T19:00:00, BUY COMPLETE, 208.70
2019-12-09T19:00:00, SELL COMPLETE, 7443.74
x + self.qty1 is 480
y + self.qty2 is 12
2019-12-09T20:00:00, SELL CREATE BCHUSDT, price = 206.88, qty = 480
2019-12-09T20:00:00, BUY CREATE BTCUSDT, price = 7300.65, qty = 12
2019-12-09T21:00:00, SELL COMPLETE, 206.96
2019-12-09T21:00:00, BUY COMPLETE, 7302.05
2019-12-10T01:00:00, CLOSE Position BCHUSDT, price = 208.15, qty = -241
2019-12-10T01:00:00, CLOSE Position BTCUSDT, price = 7371.68, qty = 6
2019-12-10T02:00:00, BUY COMPLETE, 208.15
2019-12-10T02:00:00, SELL COMPLETE, 7372.86
x + self.qty1 is 477
y + self.qty2 is 12
2019-12-13T18:00:00, SELL CREATE BCHUSDT, price = 211.82, qty = 477
2019-12-13T18:00:00, BUY CREATE BTCUSDT, price = 7239.97, qty = 12
2019-12-13T19:00:00, SELL COMPLETE, 211.78
2019-12-13T

x + self.qty1 is 418
y + self.qty2 is 12
2020-01-09T23:00:00, SELL CREATE BCHUSDT, price = 238.01, qty = 418
2020-01-09T23:00:00, BUY CREATE BTCUSDT, price = 7817.76, qty = 12
2020-01-10T00:00:00, SELL COMPLETE, 238.06
2020-01-10T00:00:00, BUY COMPLETE, 7817.74
2020-01-10T04:00:00, CLOSE Position BCHUSDT, price = 236.39, qty = -834
2020-01-10T04:00:00, CLOSE Position BTCUSDT, price = 7795.30, qty = 12
2020-01-10T05:00:00, BUY COMPLETE, 236.39
2020-01-10T05:00:00, SELL COMPLETE, 7795.33
x + self.qty1 is 408
y + self.qty2 is 12
2020-01-10T12:00:00, SELL CREATE BCHUSDT, price = 251.93, qty = 408
2020-01-10T12:00:00, BUY CREATE BTCUSDT, price = 7893.98, qty = 12
2020-01-10T13:00:00, SELL COMPLETE, 251.93
2020-01-10T13:00:00, BUY COMPLETE, 7893.98
x + self.qty1 is 392
y + self.qty2 is 12
2020-01-10T18:00:00, BUY CREATE BCHUSDT, price = 256.73, qty = 392
2020-01-10T18:00:00, SELL CREATE BTCUSDT, price = 8022.16, qty = 12
2020-01-10T19:00:00, BUY COMPLETE, 256.77
2020-01-10T19:00:00, SELL COM

x + self.qty1 is 216
y + self.qty2 is 9
2020-02-15T16:00:00, BUY CREATE BCHUSDT, price = 437.70, qty = 216
2020-02-15T16:00:00, SELL CREATE BTCUSDT, price = 9903.95, qty = 9
2020-02-15T17:00:00, BUY COMPLETE, 437.48
2020-02-15T17:00:00, SELL COMPLETE, 9903.90
2020-02-15T20:00:00, CLOSE Position BCHUSDT, price = 441.32, qty = 216
2020-02-15T20:00:00, CLOSE Position BTCUSDT, price = 9892.89, qty = -9
2020-02-15T21:00:00, SELL COMPLETE, 441.48
2020-02-15T21:00:00, BUY COMPLETE, 9893.96
x + self.qty1 is 235
y + self.qty2 is 10
2020-02-16T22:00:00, BUY CREATE BCHUSDT, price = 410.00, qty = 235
2020-02-16T22:00:00, SELL CREATE BTCUSDT, price = 9877.59, qty = 10
2020-02-16T23:00:00, BUY COMPLETE, 409.83
2020-02-16T23:00:00, SELL COMPLETE, 9877.63
2020-02-17T02:00:00, CLOSE Position BCHUSDT, price = 402.85, qty = 235
2020-02-17T02:00:00, CLOSE Position BTCUSDT, price = 9840.51, qty = -10
2020-02-17T03:00:00, SELL COMPLETE, 403.02
2020-02-17T03:00:00, BUY COMPLETE, 9840.51
x + self.qty1 is 252


x + self.qty1 is 482
y + self.qty2 is 16
2020-03-28T19:00:00, SELL CREATE BCHUSDT, price = 211.26, qty = 482
2020-03-28T19:00:00, BUY CREATE BTCUSDT, price = 6210.77, qty = 16
2020-03-28T20:00:00, SELL COMPLETE, 211.28
2020-03-28T20:00:00, BUY COMPLETE, 6210.50
2020-03-29T03:00:00, CLOSE Position BCHUSDT, price = 213.12, qty = -482
2020-03-29T03:00:00, CLOSE Position BTCUSDT, price = 6202.95, qty = 16
2020-03-29T04:00:00, BUY COMPLETE, 213.12
2020-03-29T04:00:00, SELL COMPLETE, 6203.93
x + self.qty1 is 459
y + self.qty2 is 15
2020-03-30T16:00:00, SELL CREATE BCHUSDT, price = 223.94, qty = 459
2020-03-30T16:00:00, BUY CREATE BTCUSDT, price = 6377.88, qty = 15
2020-03-30T17:00:00, SELL COMPLETE, 223.94
2020-03-30T17:00:00, BUY COMPLETE, 6377.86
2020-03-30T19:00:00, CLOSE Position BCHUSDT, price = 221.50, qty = -459
2020-03-30T19:00:00, CLOSE Position BTCUSDT, price = 6366.74, qty = 15
2020-03-30T20:00:00, BUY COMPLETE, 221.55
2020-03-30T20:00:00, SELL COMPLETE, 6366.50
x + self.qty1 is 4

2020-05-11T19:00:00, BUY COMPLETE, 223.40
2020-05-11T19:00:00, SELL COMPLETE, 8604.20
x + self.qty1 is 437
y + self.qty2 is 10
2020-05-11T19:00:00, SELL CREATE BCHUSDT, price = 233.19, qty = 437
2020-05-11T19:00:00, BUY CREATE BTCUSDT, price = 8746.86, qty = 10
2020-05-11T20:00:00, SELL COMPLETE, 233.28
2020-05-11T20:00:00, BUY COMPLETE, 8745.25
2020-05-12T04:00:00, CLOSE Position BCHUSDT, price = 234.46, qty = -25
2020-05-12T04:00:00, CLOSE Position BTCUSDT, price = 8702.57, qty = 0
2020-05-12T05:00:00, BUY COMPLETE, 234.52
x + self.qty1 is 411
y + self.qty2 is 10
2020-05-18T02:00:00, SELL CREATE BCHUSDT, price = 253.09, qty = 411
2020-05-18T02:00:00, BUY CREATE BTCUSDT, price = 9870.05, qty = 10
2020-05-18T03:00:00, SELL COMPLETE, 252.94
2020-05-18T03:00:00, BUY COMPLETE, 9870.05
2020-05-18T09:00:00, CLOSE Position BCHUSDT, price = 246.25, qty = -411
2020-05-18T09:00:00, CLOSE Position BTCUSDT, price = 9579.11, qty = 10
2020-05-18T10:00:00, BUY COMPLETE, 246.21
2020-05-18T10:00:00, S

2020-07-25T06:00:00, CLOSE Position BTCUSDT, price = 9589.01, qty = 10
2020-07-25T07:00:00, BUY COMPLETE, 241.82
2020-07-25T07:00:00, SELL COMPLETE, 9589.01
x + self.qty1 is 415
y + self.qty2 is 10
2020-07-25T09:00:00, SELL CREATE BCHUSDT, price = 242.17, qty = 415
2020-07-25T09:00:00, BUY CREATE BTCUSDT, price = 9569.38, qty = 10
2020-07-25T10:00:00, SELL COMPLETE, 242.12
2020-07-25T10:00:00, BUY COMPLETE, 9569.38
2020-07-25T10:00:00, CLOSE Position BCHUSDT, price = 242.03, qty = -415
2020-07-25T10:00:00, CLOSE Position BTCUSDT, price = 9581.70, qty = 10
2020-07-25T11:00:00, BUY COMPLETE, 241.99
2020-07-25T11:00:00, SELL COMPLETE, 9581.69
x + self.qty1 is 407
y + self.qty2 is 10
2020-07-25T16:00:00, SELL CREATE BCHUSDT, price = 247.69, qty = 407
2020-07-25T16:00:00, BUY CREATE BTCUSDT, price = 9615.51, qty = 10
2020-07-25T17:00:00, SELL COMPLETE, 247.61
2020-07-25T17:00:00, BUY COMPLETE, 9615.53
2020-07-25T19:00:00, CLOSE Position BCHUSDT, price = 247.97, qty = -407
2020-07-25T19:00:0

2020-08-21T04:00:00, SELL COMPLETE, 301.52
2020-08-21T04:00:00, BUY COMPLETE, 11866.58
2020-08-21T11:00:00, CLOSE Position BCHUSDT, price = 293.16, qty = -324
2020-08-21T11:00:00, CLOSE Position BTCUSDT, price = 11715.61, qty = 8
2020-08-21T12:00:00, BUY COMPLETE, 293.17
2020-08-21T12:00:00, SELL COMPLETE, 11715.60
x + self.qty1 is 346
y + self.qty2 is 8
2020-08-30T12:00:00, SELL CREATE BCHUSDT, price = 274.82, qty = 346
2020-08-30T12:00:00, BUY CREATE BTCUSDT, price = 11598.75, qty = 8
2020-08-30T13:00:00, SELL COMPLETE, 274.80
2020-08-30T13:00:00, BUY COMPLETE, 11598.75
2020-08-30T18:00:00, CLOSE Position BCHUSDT, price = 277.47, qty = -346
2020-08-30T18:00:00, CLOSE Position BTCUSDT, price = 11651.04, qty = 8
2020-08-30T19:00:00, BUY COMPLETE, 277.48
2020-08-30T19:00:00, SELL COMPLETE, 11651.04
x + self.qty1 is 355
y + self.qty2 is 8
2020-09-01T22:00:00, SELL CREATE BCHUSDT, price = 286.73, qty = 355
2020-09-01T22:00:00, BUY CREATE BTCUSDT, price = 11973.04, qty = 8
2020-09-01T23:00

x + self.qty1 is 443
y + self.qty2 is 8
2020-09-27T16:00:00, SELL CREATE BCHUSDT, price = 227.65, qty = 443
2020-09-27T16:00:00, BUY CREATE BTCUSDT, price = 10720.35, qty = 8
2020-09-27T17:00:00, SELL COMPLETE, 227.68
2020-09-27T17:00:00, BUY COMPLETE, 10720.36
2020-09-27T19:00:00, CLOSE Position BCHUSDT, price = 226.19, qty = -443
2020-09-27T19:00:00, CLOSE Position BTCUSDT, price = 10732.00, qty = 8
2020-09-27T20:00:00, BUY COMPLETE, 226.16
2020-09-27T20:00:00, SELL COMPLETE, 10732.01
x + self.qty1 is 438
y + self.qty2 is 8
2020-10-01T23:00:00, SELL CREATE BCHUSDT, price = 228.12, qty = 438
2020-10-01T23:00:00, BUY CREATE BTCUSDT, price = 10619.13, qty = 8
2020-10-02T00:00:00, SELL COMPLETE, 228.07
2020-10-02T00:00:00, BUY COMPLETE, 10619.13
2020-10-02T02:00:00, CLOSE Position BCHUSDT, price = 226.07, qty = -438
2020-10-02T02:00:00, CLOSE Position BTCUSDT, price = 10604.29, qty = 8
2020-10-02T03:00:00, BUY COMPLETE, 226.06
2020-10-02T03:00:00, SELL COMPLETE, 10604.29
x + self.qty1 is

In [51]:
#final results saved in endstats.csv
results = pd.read_csv('endstats_hour.csv')

In [53]:
#get best result
results[np.round(results.portfolio,2)==184478.05]

,pair1,pair2,portfolio,period
38,BCHUSDT,BTCUSDT,184478.05,264
